## Crime Rate ___ (Group 23)

#### Part 1: Introduction

#### Part 2: Methods & Results

#### 2.1: Data Exploration

Attach the libraries and set the seed:

In [1]:
# Attach the libraries.
library(tidyverse)
library(tidymodels)
library(cowplot)
library(GGally)
library(RColorBrewer)
set.seed(1000)
options(repr.plot.width = 12, repr.plot.height = 8)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.1     ✔ rsample      1.1.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.3     ✔ workflows    1.0.0
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.1     ✔ yardstick    1.1.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() ma

In [7]:
# load Grandview Woodland crime rate dataset from database
grandview_woodland_data <- read_csv("crimedata_csv_Grandview-Woodland_AllYears.csv", col_names = TRUE)
names(grandview_woodland_data) <- tolower(names(grandview_woodland_data))

#Get rid of any rows with NA
grandview_woodland_clean <- grandview_woodland_data |>
           filter_all(all_vars(!is.na(.)))

# View the cleaned dataframe.
glimpse(grandview_woodland_clean)

Rows: 48142 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): TYPE, HUNDRED_BLOCK, NEIGHBOURHOOD
dbl (7): YEAR, MONTH, DAY, HOUR, MINUTE, X, Y

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 48,142
Columns: 10
$ type          <chr> "Break and Enter Commercial", "Break and Enter Commercia…
$ year          <dbl> 2006, 2006, 2006, 2022, 2007, 2006, 2003, 2004, 2007, 20…
$ month         <dbl> 1, 11, 11, 4, 10, 4, 10, 12, 10, 10, 4, 3, 8, 11, 11, 8,…
$ day           <dbl> 30, 11, 15, 3, 15, 9, 14, 20, 1, 15, 9, 14, 1, 4, 15, 11…
$ hour          <dbl> 16, 16, 16, 0, 4, 23, 18, 4, 17, 4, 17, 5, 17, 7, 1, 5, …
$ minute        <dbl> 30, 0, 30, 0, 9, 30, 0, 26, 0, 40, 30, 21, 0, 23, 15, 20…
$ hundred_block <chr> "10XX CLARK DR", "10XX CLARK DR", "10XX CLARK DR", "10XX…
$ neighbourhood <chr> "Grandview-Woodland", "Grandview-Woodland", "Grandview-W…
$ x             <dbl> 494382.4, 494382.4, 494382.4, 494382.6, 494937.3, 494937…
$ y             <dbl> 5458077, 5458077, 5458077, 5458098, 5458069, 5458076, 54…


Figure 2.1.1: The initial Grandview Woodland dataset with the correct column names and after basic cleaning.

In [8]:
# load Shaughnessy crime rate dataset from database
shaughnessy_data <- read_csv("crimedata_csv_Shaughnessy_AllYears.csv", col_names = TRUE)
names(shaughnessy_data) <- tolower(names(shaughnessy_data))

#Get rid of any rows with NA
shaughnessy_clean <- shaughnessy_data |>
           filter_all(all_vars(!is.na(.)))

# View the cleaned dataframe.
glimpse(shaughnessy_clean)

Rows: 8530 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): TYPE, HUNDRED_BLOCK, NEIGHBOURHOOD
dbl (7): YEAR, MONTH, DAY, HOUR, MINUTE, X, Y

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 8,530
Columns: 10
$ type          <chr> "Break and Enter Commercial", "Break and Enter Commercia…
$ year          <dbl> 2022, 2022, 2022, 2018, 2019, 2021, 2021, 2021, 2022, 20…
$ month         <dbl> 2, 2, 4, 2, 6, 2, 12, 12, 4, 11, 4, 3, 8, 1, 2, 5, 5, 5,…
$ day           <dbl> 23, 25, 30, 2, 16, 15, 16, 26, 5, 28, 14, 27, 26, 28, 3,…
$ hour          <dbl> 23, 10, 22, 17, 9, 17, 5, 4, 22, 12, 14, 5, 16, 3, 5, 7,…
$ minute        <dbl> 0, 15, 45, 0, 3, 19, 33, 10, 18, 0, 35, 33, 30, 16, 17, …
$ hundred_block <chr> "10XX BALFOUR AVE", "10XX BALFOUR AVE", "10XX BALFOUR AV…
$ neighbourhood <chr> "Shaughnessy", "Shaughnessy", "Shaughnessy", "Shaughness…
$ x             <dbl> 490699.8, 490699.8, 490699.8, 490704.3, 490704.3, 490704…
$ y             <dbl> 5455444, 5455444, 5455444, 5455350, 5455350, 5455350, 54…


Figure 2.1.2: The initial Shaughnessy dataset with the correct column names and after basic cleaning.

To narrow down the focus of this study, 2012 to 2016 is chosen inclusive as the target years. Therefore, the two datas are filtered to contain only these five years of data in below. In addition, because we only need type of crime and year, only these two columns are chosen.

In [25]:
# Select only the desired years (2012-2016) and the desired columns (type of crime and year)
grandview_woodland_clean <- grandview_woodland_clean |>
    filter(year == 2012 | year == 2013 | year == 2014 | year == 2015 | year == 2016) |>
    select(type, year)

# View the filtered and selected dataframe.
glimpse(grandview_woodland_clean)

Rows: 10,900
Columns: 2
$ type <chr> "Break and Enter Commercial", "Break and Enter Commercial", "Brea…
$ year <dbl> 2015, 2015, 2015, 2016, 2016, 2016, 2015, 2015, 2016, 2012, 2016,…


Figure 2.1.3: The complete Grandview Woodland dataset with the correct column filtered and selected.

In [10]:
# Select only the desired years (2012-2016) and the desired columns (type of crime and year)
shaughnessy_clean <- shaughnessy_clean |>
    filter(year == 2012 | year == 2013 | year == 2014 | year == 2015 | year == 2016) |>
    select(type, year)

# View the filtered and selected dataframe.
glimpse(shaughnessy_clean)

Rows: 2,093
Columns: 2
$ type <chr> "Break and Enter Commercial", "Break and Enter Commercial", "Brea…
$ year <dbl> 2014, 2016, 2013, 2013, 2014, 2014, 2012, 2013, 2015, 2015, 2014,…


Figure 2.1.4: The complete Shaughnessy dataset with the correct column filtered and selected.

From the two cleaned datasets, we obtained that Grandview Woodland has a total of 10,900 crimes in the year of 2012 to 2016 while Shaughnessy has a total of 2,093 crimes in the year of 2012 to 2016. 

# DATA WRANGLING FOR OURSELF:


In [24]:
g <- grandview_woodland_clean |>
    filter(type == "Break and Enter Residential/Other") |>
    summarize(n=n()) |>
    mutate(z = n/29175) |>
    pull(z)

g

s <- shaughnessy_clean |>
    filter(type == "Break and Enter Residential/Other") |>
    summarize(n=n()) |>
    mutate(z = n/8430) |>
    pull(z)

s

[1] 0.03002571

[1] 0.06809015

#### 2.2: Data Analysis